In [1]:
import torch # 导入pytorch
from torch import nn, optim # 导入神经网络与优化器对应的类
import torch.nn.functional as F
from torchvision import datasets, transforms ## 导入数据集与数据预处理的方法

data_dir = '../../../wenrj/datasets/'

# 定义一个转换器：标准化图像数据，使得灰度数据在-1到+1之间，是的训练出的权值在0附近，利于神经网络的训练
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])

# 下载Fashion-MNIST训练集数据，并构建训练集数据载入器trainloader,每次从训练集中载入64张图片，每次载入都打乱顺序
#shuffle:每次重新打乱进行抽取
batch_size=64
trainset = datasets.FashionMNIST(root=data_dir, download=True,
train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,
batch_size=batch_size, shuffle=True)
# 下载Fashion-MNIST测试集数据，并构建测试集数据载入器trainloader,每次从测试集中载入64张图片，每次载入都打乱顺序
testset = datasets.FashionMNIST(root=data_dir, download=True,
train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset,
batch_size=64, shuffle=True)

#打开数据集中的图片
images, labels = next(iter(trainloader))
images.shape

torch.Size([64, 1, 28, 28])

import matplotlib.pyplot as plt
plt.imshow(image.reshape(28,28))
labellist = ['T恤','裤子','套衫','裙子','外套','凉鞋','汗衫','运动鞋','包包','靴子']
print(f'这张图片对应的标签是 {labellist[label]}')

#搭建并训练四层全连接神经网络
from torch import nn, optim
import torch.nn.functional as F
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256) #第一层:256个神经元
        self.fc2 = nn.Linear(256, 128) #第二层:128个神经元
        self.fc3 = nn.Linear(128, 64) #第三层:64个神经元
        self.fc4 = nn.Linear(64, 10) #输出层


    def forward(self, x):
        '''
        1.表示将张量 x 重新构造为一个新的张量，其中第一维度的大
        小保持不变，而后面的维度被拉成一个扁平的向量
        2.x.shape[0]表示batch_size,即一个训练批次的数量
        '''
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.fc4(x)
        x = F.log_softmax((x), dim=1)
        return x

# 对上面定义的Classifier类进行实例化
model = Classifier()
model.to('cuda') #如果不是gpu版本的pytorch，将这行注释掉

# 定义损失函数为交叉熵
criterion = nn.CrossEntropyLoss()
# 优化方法为SDG，学习率为0.003
optimizer = optim.SGD(model.parameters(), lr=0.003)
# 对训练集的全部数据学习15遍，这个数字越大，训练时间越长
epochs = 15
# 将每次训练的训练误差和测试误差存储在这两个列表里，后面绘制误差变化折线图用
train_losses, test_losses = [], []
def train():
    print('开始训练:')
    for e in range(epochs):
        running_loss = 0
        # 对训练集中的所有图片都过一遍
        for images, labels in trainloader:
            images = images.cuda() #如果不是gpu版本的pytorch，将这行注释掉
            labels = labels.cuda() #如果不是gpu版本的pytorch，将这行注释掉
            # 将优化器中的求导结果都设为0，否则会在每次反向传播之后叠加之前的
            optimizer.zero_grad()
            # 对64张图片进行推断，计算损失函数，反向传播优化权重，将损失求和
            log_ps = model(images)
            loss = criterion(log_ps, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

    # 每次学完一遍数据集，都进行以下测试操作
    else:
        test_loss = 0
        accuracy = 0
        # 测试的时候不需要开自动求导和反向传播
        with torch.no_grad():
        # 将模型转换为评估模式，在该模式下不会影响到训练
        model.eval()
        # 对测试集中的所有图片都过一遍
        for images, labels in testloader:images = images.cuda() #如果不是gpu版本的pytorch，将这行注释掉
        labels = labels.cuda() #如果不是gpu版本的pytorch，将这行注释掉
        # 对传入的测试集图片进行正向推断、计算损失，accuracy为测试集一万张图片中模型预测正确率
        log_ps = model(images)
        test_loss += criterion(log_ps, labels)
        '''这里代码有疑问'''
        ps = torch.exp(log_ps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class ==
        labels.view(*top_class.shape)
        # 等号右边为每一批64张测试图片中预测正确的占比
        accuracy +=torch.mean(equals.type(torch.FloatTensor))
        '''这里代码有疑问'''
        # 恢复Dropout
        model.train()
        # 将训练误差和测试误差存在两个列表里，后面绘制误差变化折线图用
        train_losses.append(running_loss/len(trainloader))

        test_losses.append(test_loss/len(testloader))
        print("训练集学习次数: {}/{}.. ".format(e+1,epochs),"训练误差: {:.3f}..".format(running_loss/len(trainloader)),"测试误差: {:.3f}..".format(test_loss/len(testloader)),"模型分类准确率:{:.3f}".format(accuracy/len(testloader)))

#验证模型效果
#绘制训练误差和测试误差随学习次数增加的变化
import matplotlib.pyplot as plt

# 将列表中的所有张量从 GPU 移动到 CPU 上
for i, tensor in enumerate(test_losses):
    test_losses[i] = tensor.cpu()

plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend()



IndentationError: expected an indented block (158717445.py, line 100)